In [1]:
from rdflib import Graph

In [2]:
# Loading OWL ontologies

cmt_ontology = Graph()
ekaw_ontology = Graph()

# Load ontologies from OWL files

cmt_ontology.parse("cmt.owl")
ekaw_ontology.parse("ekaw.owl")

<Graph identifier=N4b3a65ae0dbf40988704dc4622c88411 (<class 'rdflib.graph.Graph'>)>

In [3]:
# Extracting concepts from cmt.owl
cmt_concepts = set()
for subject, predicate, obj in cmt_ontology.triples((None, None, None)):
    if isinstance(obj, (str, int, float)):
        cmt_concepts.add(obj)
        
# Extracting concepts from ekaw.owl
ekaw_concepts = set()
for subject, predicate, obj in ekaw_ontology.triples((None, None, None)):
    if isinstance(obj, (str, int, float)):
        ekaw_concepts.add(obj)

In [4]:
# Storage of concepts in text files
with open("cmt.txt", "w") as cmt_file:
    cmt_file.write("\n".join(cmt_concepts))

with open("ekaw.txt", "w") as ekaw_file:
    ekaw_file.write("\n".join(ekaw_concepts))

print("Concepts from cmt.owl extracted and stored in cmt.txt")
print("Concepts from ekaw.owl extracted and stored in ekaw.txt")

Concepts from cmt.owl extracted and stored in cmt.txt
Concepts from ekaw.owl extracted and stored in ekaw.txt


In [5]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [6]:
# Loading concept files
with open("cmt.txt", "r") as cmt_file:
    cmt_concepts = cmt_file.read().splitlines()

with open("ekaw.txt", "r") as ekaw_file:
    ekaw_concepts = ekaw_file.read().splitlines()

In [7]:
# Creation of a Word2Vec model combining the concepts of cmt and ekaw
combined_concepts = cmt_concepts + ekaw_concepts
model = Word2Vec(sentences=[combined_concepts], vector_size=100, window=5, min_count=1, sg=0)

In [8]:
# Correspondence of vectors between cmt and ekaw
matching_vectors = {}
for cmt_concept in cmt_concepts:
    if cmt_concept in model.wv.key_to_index:
        cmt_vector = model.wv[cmt_concept]
        most_similar_ekaw = model.wv.most_similar([cmt_vector], topn=1)
        matching_vectors[cmt_concept] = most_similar_ekaw[0]

In [9]:
# Displaying corresponding vectors 1.1 matching
for cmt_concept, ekaw_concept in matching_vectors.items():
    print(f"CMT Concept: {cmt_concept}")
    print(f"EKAW Concept: {ekaw_concept[0]}")
    print(f"Similarity Score: {ekaw_concept[1]}")
    print()

CMT Concept: http://cmt#Acceptance
EKAW Concept: http://cmt#Acceptance
Similarity Score: 0.9999999403953552

CMT Concept: A special type of Reviewer.  There can be any number of Reviewers for a given paper, but only one Meta-Reviewer.  The Meta-Reviewer can go over all the reviews submitted for the paper and submit their own review.  Questions for the Meta-Reviewer can be different from those for a normal Reviewer.  Their role is to ensure that the reviews are good and consistent. The decision to use Meta-Reviewers is optional, and must be set before paper assignment occurs.
EKAW Concept: A special type of Reviewer.  There can be any number of Reviewers for a given paper, but only one Meta-Reviewer.  The Meta-Reviewer can go over all the reviews submitted for the paper and submit their own review.  Questions for the Meta-Reviewer can be different from those for a normal Reviewer.  Their role is to ensure that the reviews are good and consistent. The decision to use Meta-Reviewers is op

In [14]:
# Correspondence of vectors between cmt and ekaw (many to many)
matching_vectors = {}
for cmt_concept in cmt_concepts:
    if cmt_concept in model.wv.key_to_index:
        cmt_vector = model.wv[cmt_concept]
        most_similar_ekaw = model.wv.most_similar([cmt_vector], topn=3)  # Top 3 correspondances
        matching_vectors[cmt_concept] = most_similar_ekaw

In [15]:
# Displaying corresponding vectors
for cmt_concept, ekaw_concepts in matching_vectors.items():
    print(f"CMT Concept: {cmt_concept}")
    print("EKAW Concepts:")
    for ekaw_concept in ekaw_concepts:
        print(f"  - {ekaw_concept[0]} (Similarity Score: {ekaw_concept[1]})")
    print()

CMT Concept: http://cmt#Acceptance
EKAW Concepts:
  - http://cmt#Acceptance (Similarity Score: 0.9999999403953552)
  - N067eae3fdfe54fe98e8512ff43880835 (Similarity Score: 0.26777568459510803)
  - http://cmt#Reviewer (Similarity Score: 0.23839528858661652)

CMT Concept: A special type of Reviewer.  There can be any number of Reviewers for a given paper, but only one Meta-Reviewer.  The Meta-Reviewer can go over all the reviews submitted for the paper and submit their own review.  Questions for the Meta-Reviewer can be different from those for a normal Reviewer.  Their role is to ensure that the reviews are good and consistent. The decision to use Meta-Reviewers is optional, and must be set before paper assignment occurs.
EKAW Concepts:
  - A special type of Reviewer.  There can be any number of Reviewers for a given paper, but only one Meta-Reviewer.  The Meta-Reviewer can go over all the reviews submitted for the paper and submit their own review.  Questions for the Meta-Reviewer can 